## 1. Datenaufbereitung (Preprocessing)

In [ ]:
import pandas as pd

# Liste der Dateipfade mit dem relativen Pfad '../'
file_paths = [
    '../data/audi.csv',
    '../data/bmw.csv',
    '../data/ford.csv',
    '../data/hyundi.csv',
    '../data/mercedes.csv',
    '../data/skoda.csv',
    '../data/toyota.csv',
    '../data/vauxhall.csv',
    '../data/vw.csv',
]

# Liste, um alle DataFrames zu speichern
all_dataframes = []

# Durchlaufe alle Dateipfade und lade die CSV-Dateien
for file_path in file_paths:
    df = pd.read_csv(file_path)
    brand = file_path.split('/')[-1].split('.')[0]  # Extrahiere die Marke aus dem Dateinamen
    df['brand'] = brand  # Füge eine Spalte mit der Marke hinzu
    # Ändere die Reihenfolge der Spalten, um "brand" an den Anfang zu setzen
    df = df[['brand'] + [col for col in df.columns if col != 'brand']]
    all_dataframes.append(df)

# Verbinde alle DataFrames
df = pd.concat(all_dataframes, ignore_index=True)

In [ ]:
df.head()

### Schritt 1: Überprüfung auf Duplikate

In [ ]:
df.shape

In [ ]:
# Überprüfung auf Duplikate
duplicate_count = df.duplicated().sum()
print(duplicate_count)

In [ ]:
# Entfernen von Duplikaten, falls vorhanden
if duplicate_count > 0:
    data = df.drop_duplicates()

### Schritt 2: Prüfung auf fehlende Werte

In [ ]:
# Überprüfung auf fehlende Werte in jeder Spalte
missing_values = data.isnull().sum()
missing_values[missing_values > 0]

Es gibt fehlende Werte in den Spalten tax, mpg, und tax(£). Die Spalte tax(£) scheint in einem großen Teil des Datensatzes zu fehlen. Wir könnten diese Spalte entfernen, da sie nicht viele Informationen enthält. Bei den anderen Spalten könnten wir die fehlenden Werte mit geeigneten Methoden behandeln, wie z.B. durch das Ausfüllen mit dem Durchschnittswert oder Median.

Entfernen der Spalte tax(£) und Behandeln von fehlenden Werten

In [ ]:
# Entfernen der Spalte "tax(£)"
df = data.drop(columns=['tax(£)'])

# Fehlende Werte in den Spalten "tax" und "mpg" mit dem Median auffüllen
df['tax'].fillna(df['tax'].median(), inplace=True)
df['mpg'].fillna(df['mpg'].median(), inplace=True)

# Überprüfung, ob noch fehlende Werte vorhanden sind
df.isnull().sum()

In [ ]:
print(df['year'].min())
print(df['year'].max())

In [ ]:
# Entfernen der Zeilen, in denen das Jahr 1970 oder 2060 ist
df = df[(df['year'] != 1970) & (df['year'] != 2060)]

# Überprüfung der neuen Form des Datensatzes
df.shape

In [ ]:
df['transmission'].unique()

In [ ]:
df

In [ ]:
unique_values_counts = df['fuelType'].value_counts()

print("Einzelne eindeutige Werte und ihre Häufigkeit:")
print(unique_values_counts)

In [ ]:
unique_values_counts = df['transmission'].value_counts()

print("Einzelne eindeutige Werte und ihre Häufigkeit:")
print(unique_values_counts)

In [ ]:
# Function to convert miles per gallon (mpg) to liters per 100 kilometers (l/100km)
def mpg_to_l_per_100km(mpg):
    return 235.215 / mpg

# Function to convert miles to kilometers
def miles_to_km(miles):
    return miles * 1.60934

# Translating column names
df.columns = [
    'Marke', 'Modell', 'Baujahr', 'Preis (£)', 'Getriebe', 'Kilometerstand', 'Kraftstoffart', 'Steuer (£)', 'Verbrauch (l/100km)', 'Motorgröße (l)']

# Translating values in the 'Getriebe' column
df['Getriebe'] = df['Getriebe'].map({'Manual': 'Manuell', 
                                     'Automatic': 'Automatik', 
                                     'Semi-Auto': 'Halbautomatisch'})

# Translating values in the 'Kraftstoffart' column
df['Kraftstoffart'] = df['Kraftstoffart'].map({'Petrol': 'Benzin', 
                                               'Diesel': 'Diesel', 
                                               'Hybrid': 'Hybrid',
                                               'Other': 'Andere',
                                               'Electric': 'Elektrisch'})

# Converting mileage from miles to kilometers
df['Kilometerstand'] = df['Kilometerstand'].apply(miles_to_km).astype(int)

# Converting mpg to liters per 100 kilometers
df['Verbrauch (l/100km)'] = df['Verbrauch (l/100km)'].apply(mpg_to_l_per_100km).round(2)

# Capitalizing the first letter of each entry in the "Marke" column
df['Marke'] = df['Marke'].str.capitalize()


# Displaying the first few rows after rounding
df.head()


In [ ]:
# Wechselkurs von Pfund zu Euro festlegen
exchange_rate = 1.15

# Umrechnung der Preise und Steuern von Pfund in Euro
df['Preis (€)'] = df['Preis (£)'] * exchange_rate
df['Steuer (€)'] = df['Steuer (£)'] * exchange_rate

df['Preis (€)']


# Entfernen der alten Spalten in Pfund
df.drop(columns=['Preis (£)', 'Steuer (£)'], inplace=True)

# Anzeige der ersten 5 Zeilen des aktualisierten Datensatzes
df.head()


In [ ]:
df.shape

In [ ]:

# Die Spalten auswählen, in denen Leerzeichen entfernt werden sollen
string_columns = ['Marke', 'Modell', 'Getriebe', 'Kraftstoffart']

# Leerzeichen in den ausgewählten Spalten entfernen
df[string_columns] = df[string_columns].apply(lambda x: x.str.replace(" ", ""))

In [ ]:
df.to_csv('../data (clean)/df_clean.csv', index=False)